In [1]:
%matplotlib inline

import os
import pandas as pd
import geopandas as gpd
import importlib.util
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import gdal
import numpy as np
import rasterio
import glob
from IPython.display import display

plt.style.use("ggplot")

In [2]:
# Import CL functions
spec = importlib.util.spec_from_file_location(
    "critical_loads", "../../notebooks/critical_loads.py"
)
cl = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cl)

In [3]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


In [4]:
# Connect to CL db
cl_eng = nivapy.da.connect_postgis(database="critical_loads")

Connection successful.


# CL Vestland: Calculate exceedances

**Note:** This notebook is rather messy and should be tidied up. In particular, cell 6 currently subsets the data to avoid issues with overlapping catchment polygons: the three non-overlapping polygons are processed simultaneously, and the fourth is then processed separately. This **requires running the notebook twice** (once for each catchment "subset"), each time changing the name of the summary CSV generated in section 5 to avoid overwriting. This is "hacky" and can be cleaned up, but it seems pragmatic for the moment.

## 1. Get catchments of interest

### 1.1. Outflow points

In [5]:
# Get outflows
sql = (
    "SELECT * FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id FROM niva.projects_stations "
    "  WHERE project_id IN ( "
    "    SELECT project_id FROM niva.projects "
    "    WHERE project_name = 'CL Vestland' "
    "    ) "
    "  ) "
)
stn_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
stn_gdf = stn_gdf.to_crs({"init": "epsg:25833"})

stn_gdf.head()

/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,station_id,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,87,Sam_Froe,Samnanger_Frolandselva,None,5.798510,60.380058,POINT (-6218.205 6729188.752)
1,86,Sam_Stor,Samnanger_Storelva,None,5.798443,60.384011,POINT (-6160.286 6729626.599)
2,84,Sam_Tyss,Samnanger_Tysseelva,None,5.758580,60.374899,POINT (-8485.813 6728926.678)
3,85,Sam_Tyss_IntCat,Samnanger_Tysseelva_Intercatchment,None,5.758580,60.374899,POINT (-8485.813 6728926.678)
4,88,Sam_Frok,Samnanger_Frolandskanalen,None,5.793918,60.381434,POINT (-6448.257 6729376.408)


In [6]:
# Subset
#stn_gdf = stn_gdf.query("station_id in (86, 87, 88)")
stn_gdf = stn_gdf.query("station_id == 84")

### 1.2. Catchment boundaries

In [7]:
# Get catchments
stn_list = list(stn_gdf["station_id"].astype(str))
bind_pars = ",".join(stn_list)
sql = f"SELECT * FROM niva.catchments " f"WHERE station_id IN ({bind_pars})"
cat_gdf = gpd.read_postgis(sql, eng)

# Reproject to ETRS89 UTM Z33N
cat_gdf = cat_gdf.to_crs({"init": "epsg:25833"})

# Join codes
cat_gdf = cat_gdf.merge(
    stn_gdf[["station_id", "station_code"]], how="left", on="station_id"
)
cat_gdf = cat_gdf[["station_id", "station_code", "geom"]]

cat_gdf.head()

/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,station_id,station_code,geom
0,84,Sam_Tyss,"MULTIPOLYGON (((547.072 6746299.978, 595.735 6..."


## 2. Calculate critical loads

## 2.1. Process input template

Kari wishes to use the "F-factor" method for these calculations.

In [8]:
# Path to completed template
xl_path = r"../input_template_critical_loads_water_Samnanger.xlsx"

bc0_method = "_Ffac"

# Calculate CLs
cl_df = cl.calculate_critical_loads_for_water(xl_path)

# Get cols of interest
cols = [
    "Region_id",
    f"CLAOAA{bc0_method}_meq/m2/yr",
    "ENO3_flux_meq/m2/yr",
    "CLminN_meq/m2/yr",
    f"CLmaxNoaa{bc0_method}_meq/m2/yr",
    f"CLmaxSoaa{bc0_method}_meq/m2/yr",
]
cl_df = cl_df[cols]

cl_df

,Region_id,CLAOAA_Ffac_meq/m2/yr,ENO3_flux_meq/m2/yr,CLminN_meq/m2/yr,CLmaxNoaa_Ffac_meq/m2/yr,CLmaxSoaa_Ffac_meq/m2/yr
0,Sam_Tyss,49.462026,28.416200,3.170623,61.844542,49.828589
1,Sam_Stor,117.963426,41.068558,3.538674,135.547045,118.053337
2,Sam_Froe,58.437760,48.473570,3.382475,70.294610,58.632974
3,Sam_Frok,30.957585,24.210766,2.994525,40.751515,31.290189


## 2.2. Rasterise critical loads

In [9]:
# Cell size for rasterisation
cell_size = 50

# Snap tiff
snap_tif = f"/home/jovyan/projects/critical_loads_2/cl_vestland/raster/cl_vestland_snap_ras_{cell_size}m.tif"

# Simplify col names (as units are consistent)
cl_df.columns = [i.split("_")[0].lower() for i in cl_df.columns]
cl_df.rename({"region": "station_code"}, inplace=True, axis=1)
cl_df.dropna(how="any", inplace=True)

# Add CLminS as 0
cl_df["clmins"] = 0

# Join to catchments
cat_gdf = cat_gdf.merge(cl_df, how="left", on="station_code")

# Save temporary file
temp = "../raster/temp.geojson"
cat_gdf.to_file(temp, driver="GeoJSON")

# Rasterize each column
cols = ["claoaa", "eno3", "clminn", "clmaxnoaa", "clmaxsoaa", "clmins"]
for col in cols:
    print(f"Rasterising {col}...")
    # Tiff to create
    out_tif = f"/home/jovyan/projects/critical_loads_2/cl_vestland/raster/critical_loads/{col}_meqpm2pyr_{cell_size}m.tif"
    cl.vec_to_ras(temp, out_tif, snap_tif, col, -9999, "Float32")

# Delete temp file
os.remove(temp)

cat_gdf.head()

Rasterising claoaa...
Rasterising eno3...
Rasterising clminn...
Rasterising clmaxnoaa...
Rasterising clmaxsoaa...
Rasterising clmins...


,station_id,station_code,geom,claoaa,eno3,clminn,clmaxnoaa,clmaxsoaa,clmins
0,84,Sam_Tyss,"MULTIPOLYGON (((547.072 6746299.978, 595.735 6...",49.462026,28.4162,3.170623,61.844542,49.828589,0


## 3. Process deposition data

### 3.1. Select deposition series

In [10]:
# List available series
with pd.option_context("display.max_colwidth", -1):
    ser_grid = cl.view_dep_series(cl_eng)
    display(ser_grid)

,series_id,name,short_name,grid,description
0,1,Middel 1978-1982,7882,blr,Fordelt til BLR av NILU 2002
1,2,Middel 1992-1996,9296,blr,Fordelt til BLR av NILU 2002
2,3,Middel 1997-2001,9701,blr,Fordelt til BLR av NILU 2002
3,4,Middel 2002-2006,0206,blr,Fordelt til BLR av NILU 2008 (Wenche Aas)
4,5,Beregnet 2010,None,None,"Gøteborg protokollen 1999. ""Gamle"""
5,6,Middel 1983-1987,None,None,Tall fra den gamle tålegrensebasen(D_S/N). OBS Verdier omgjort fra g til mg
6,7,Middel 1988-1992,None,None,Tall fra den gamle tålegrensebasen(D-S90) NB kun S.OBS Verdier omgjort fra g til mg
7,8,"2010 (EMEP/CCE 2008) ""his""",None,None,NB: konvertert fra ekv/ha/yr til mekv/m2/yr (/10). 24.10.1010. TOH Her er det noe rart
8,9,"2020 (EMEP/CCE 2008) ""MFR""",None,None,NB: konvertert fra ekv/ha/yr til mekv/m2/yr (/10) 24.10.1010. TOH Her er det noe rart
9,10,"2020 (EMEP/CCE 2008) ""CLE""",None,None,NB: konvertert fra ekv/ha/yr til mekv/m2/yr (/10) 24.10.1010. TOH Her er det noe rart


We are interested in series IDs 28 and 59. Series ID 57 will also be required for bias correction.

### 3.2. Rasterise deposition data

In [11]:
ser_dict = {"1216": 28, "2010": 57, "2030": 59}

for par in ["nitrogen", "sulphur"]:
    for period in ser_dict.keys():
        print(f"Rasterising {par}, {period}...")
        ser_id = ser_dict[period]

        # Get dep data
        dep_gdf = cl.extract_deposition_as_gdf(
            ser_id, par, cl_eng, veg_class="grid average"
        ).to_crs({"init": "epsg:25833"})

        # Save temporary file
        temp = "../raster/temp.geojson"
        dep_gdf.to_file(temp, driver="GeoJSON")

        # Convert to raster
        col_name = f"{par[0]}dep_meqpm2pyr"
        out_tif = f"/home/jovyan/projects/critical_loads_2/cl_vestland/raster/deposition/{par[0]}dep_{period}_meqpm2pyr_{cell_size}m.tif"
        cl.vec_to_ras(temp, out_tif, snap_tif, col_name, -9999, "Float32")

        # Delete temp file
        os.remove(temp)

Rasterising nitrogen, 1216...


/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Rasterising nitrogen, 2010...


/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Rasterising nitrogen, 2030...


...

/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Rasterising sulphur, 2010...


/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Rasterising sulphur, 2030...


/home/jovyan/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


### 3.3. Bias-correct deposition data

In [12]:
# Output location
data_fold = r"../raster/deposition"

# Loop over pars
for par in ["nitrogen", "sulphur"]:
    # Paths
    base_path = os.path.join(data_fold, f"{par[0]}dep_1216_meqpm2pyr_{cell_size}m.tif")
    emep2010_path = os.path.join(
        data_fold, f"{par[0]}dep_2010_meqpm2pyr_{cell_size}m.tif"
    )
    emep2030_path = os.path.join(
        data_fold, f"{par[0]}dep_2030_meqpm2pyr_{cell_size}m.tif"
    )

    # Datasets
    base_src = rasterio.open(base_path)
    emep2010_src = rasterio.open(emep2010_path)
    emep2030_src = rasterio.open(emep2030_path)

    # Create output dataset
    prof = base_src.profile
    out_tif = os.path.join(data_fold, f"{par[0]}dep_2030bc_meqpm2pyr_{cell_size}m.tif")
    out_dst = rasterio.open(out_tif, "w", **prof)

    # Process in blocks to conserve memory
    for block_index, window in base_src.block_windows(1):
        # Read block
        base_block = base_src.read(window=window).astype(float)
        emep2010_block = emep2010_src.read(window=window).astype(float)
        emep2030_block = emep2030_src.read(window=window).astype(float)

        # Set NoData to NaN
        base_block[base_block == -9999] = np.nan
        emep2010_block[emep2010_block == -9999] = np.nan
        emep2030_block[emep2030_block == -9999] = np.nan

        # Apply delta-change method
        pred2030_block = base_block * emep2030_block / emep2010_block

        # Set NaN to -9999
        pred2030_block = np.nan_to_num(pred2030_block, nan=-9999)

        out_dst.write(pred2030_block.astype(rasterio.float32), window=window)

    # Tidy up
    base_src.close()
    emep2010_src.close()
    emep2030_src.close()
    out_dst.close()

## 4. Calculate exceedances

### 4.1. SSWC

In [13]:
for period in ["1216", "2030bc"]:
    # Read grids
    s_tif = f"../raster/deposition/sdep_{period}_meqpm2pyr_{cell_size}m.tif"
    s_dep, s_ndv, epsg, extent = nivapy.spatial.read_raster(s_tif)

    eno3_tif = f"../raster/critical_loads/eno3_meqpm2pyr_{cell_size}m.tif"
    eno3fl, eno3_ndv, epsg, extent = nivapy.spatial.read_raster(eno3_tif)

    claoaa_tif = f"../raster/critical_loads/claoaa_meqpm2pyr_{cell_size}m.tif"
    claoaa, cla_ndv, epsg, extent = nivapy.spatial.read_raster(claoaa_tif)

    # Set ndv
    s_dep[s_dep == s_ndv] = np.nan
    eno3fl[eno3fl == eno3_ndv] = np.nan
    claoaa[claoaa == cla_ndv] = np.nan

    # Exceedance
    sswc_ex = s_dep + eno3fl - claoaa
    del s_dep, eno3fl, claoaa

    # Set <0 to 0
    sswc_ex[sswc_ex < 0] = 0

    # Write geotif
    sswc_tif = f"../raster/exceedance/sswc_ex_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(sswc_ex, sswc_tif, snap_tif, -1, gdal.GDT_Float32)
    del sswc_ex

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in less


### 4.2. FAB

In [14]:
# Read CL arrays
for period in ["1216", "2030bc"]:
    array_dict = {}

    for name in ["clminn", "clmaxnoaa", "clmins", "clmaxsoaa"]:
        # Read tif
        tif_path = f"../raster/critical_loads/{name}_meqpm2pyr_{cell_size}m.tif"
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
        data[data == ndv] = np.nan
        array_dict[name] = data

    # Read dep arrays
    for name in ["ndep", "sdep"]:
        # Read tif
        tif_path = f"../raster/deposition/{name}_{period}_meqpm2pyr_{cell_size}m.tif"
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
        data[data == ndv] = np.nan
        array_dict[name] = data

    # Extract arrays from dict
    cln_min = array_dict["clminn"]
    cln_max = array_dict["clmaxnoaa"]
    cls_min = array_dict["clmins"]
    cls_max = array_dict["clmaxsoaa"]
    dep_n = array_dict[f"ndep"]
    dep_s = array_dict[f"sdep"]

    # Estimate exceedances
    ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
        cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
    )

    # Save GeoTiffs
    # N
    n_tif = f"../raster/exceedance/fab_ex_n_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

    # S
    s_tif = f"../raster/exceedance/fab_ex_s_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

    # N+S
    ns_tif = f"../raster/exceedance/fab_ex_ns_{period}_meqpm2pyr_{cell_size}m.tif"
    cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

    # Exceedance 'region'
    reg_tif = f"../raster/exceedance/fab_ex_reg_id_{period}_{cell_size}m.tif"
    cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)

../../notebooks/critical_loads.py:1567: RuntimeWarning: invalid value encountered in less
  mask = (cln_min < 0) | (cln_max < 0) | (cls_min < 0) | (cls_max < 0)
../../notebooks/critical_loads.py:1594: RuntimeWarning: invalid value encountered in less_equal
  & (edited == 0)
../../notebooks/critical_loads.py:1602: RuntimeWarning: invalid value encountered in less_equal
  mask = (dep_s <= cls_min) & (edited == 0)
../../notebooks/critical_loads.py:1609: RuntimeWarning: invalid value encountered in less_equal
  mask = (dep_n <= cln_min) & (edited == 0)
../../notebooks/critical_loads.py:1616: RuntimeWarning: invalid value encountered in greater_equal
  mask = (-(dep_n - cln_max) * dn >= (dep_s - cls_min) * ds) & (edited == 0)
../../notebooks/critical_loads.py:1623: RuntimeWarning: invalid value encountered in less_equal
  mask = (-(dep_n - cln_min) * dn <= (dep_s - cls_max) * ds) & (edited == 0)


## 5. Summary statistics

In [15]:
# Save temporary file
temp = "../raster/temp.geojson"
cat_gdf.to_file(temp, driver="GeoJSON")

# Get paths to all dep and ex grids
search_path1 = "../raster/exceedance/*.tif"
flist1 = glob.glob(search_path1)
search_path2 = "../raster/deposition/*.tif"
flist2 = glob.glob(search_path2)
search_path3 = "../raster/critical_loads/*.tif"
flist3 = glob.glob(search_path3)
flist = flist1 + flist2 + flist3

df_list = []
for fname in flist:
    ds_name = os.path.split(fname)[1][:-4]
    sum_df = nivapy.spatial.zonal_stats(temp, fname, -9999, global_src_extent=True)
    sum_df["dataset"] = ds_name
    sum_df = pd.concat([sum_df, cat_gdf[["station_code"]]], axis=1, sort=True)
    df_list.append(sum_df)

sum_df = pd.concat(df_list, sort=True)
sum_df = sum_df[["dataset", "station_code", "min", "mean", "max", "std", "count"]]

os.remove(temp)

# Save
out_csv = r"../output/cl_vestland_results_summary_meqpm2pyr_2.csv"
sum_df.to_csv(out_csv, index=False)

In [16]:
sum_df.head()

,dataset,station_code,min,mean,max,std,count
0,fab_ex_n_2030bc_meqpm2pyr_50m,Sam_Tyss,0.0,0.000000,0.0,0.000000,94510
0,sswc_ex_2030bc_meqpm2pyr_50m,Sam_Tyss,0.0,0.000000,0.0,0.000000,94510
0,fab_ex_s_2030bc_meqpm2pyr_50m,Sam_Tyss,0.0,0.000000,0.0,0.000000,94510
0,fab_ex_reg_id_1216_50m,Sam_Tyss,0.0,2.449868,3.0,1.160927,94510
0,fab_ex_ns_2030bc_meqpm2pyr_50m,Sam_Tyss,0.0,0.000000,0.0,0.000000,94510
